In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Input,GlobalMaxPool1D, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive//My Drive/Andmeteadus/Masinõpe/4N/")

Mounted at /content/drive


In [ ]:
data_path = "cleaned_am_nam_articles.csv"
data = pd.read_csv(data_path)
data = data.dropna()
train, test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=True)
train_articles = train["article"]
train_labels = train["label"]
test_articles = test["article"]
test_labels = test["label"]

le = LabelEncoder()
le.fit(test_labels)
y_train = le.transform(train_labels)
y_test = le.transform(test_labels)
y_train_vec = to_categorical(y_train)
y_test_vec = to_categorical(y_test)

In [ ]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig, DistilBertTokenizer, TFDistilBertModel

model_name = 'distilbert-base-uncased'

config = DistilBertConfig(num_labels=2)
id2label = {0:'nam', 1:'am'}
config.id2label = id2label
config.output_hidden_states = False
# Mudel ise
transformer_model = TFDistilBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
max_length = 128
tokenizer = DistilBertTokenizer.from_pretrained(model_name, do_lower_case=True, add_special_tokens=True, max_length=max_length, pad_to_max_length=True)

X_train = list(np.array(train_articles))
X_test = list(np.array(test_articles))
# Sisendiks on tokeniseerijalt saadud sõned,
# aga võib anda ka maski: 1, kui tegu on sisulise sõnega ja 0, kui tegu on puhvrinulliga
X_train_bert = tokenizer(
    text=X_train,
    add_special_tokens=True, #[CLS] jms
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True)
X_test_bert = tokenizer(
    text=X_test,
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True)

In [ ]:
# Sisu näited:
print("Sisendiks on:",[x for x in X_train_bert])
print(X_train_bert['input_ids'][0])
print(X_train_bert['attention_mask'][0])

Sisendiks on: ['input_ids', 'attention_mask']
tf.Tensor(
[  101  8273 23153  1010  3983  2255  1010 12609  1006 11333  2213  1007
  1011  1011  1996  3757  1997  3425  2038  2170  2588  2375  9786  2000
  2202  2035  4072  5761  2000  5676  3424  1011  2769 28289  1998  3424
  1011  9452 12135 14422  2024  9768  1012  2375  9786  2020  2036  2356
  2000  2202  1996  4072  3423  5761  2114 19591  2104  2976 10037  2322
  1997  2760  1999  7189  2000  2769 28289  1010  2049  3237  7040  1010
  1998  7882 18645  6567  2004  2112  1997  1996 17641  1005  1055  4073
  2000  4769  1996 10831  3378  2007  3361  6997  1012  1996  3757  7902
  2098  1996  5197  1997  2375  9786  1005  2535  1999  4337  2075  2769
 28289  1010  2004  2092  2004  1996  2342  2005  9559  2000  7868  2037
  2535  1999  3554  2023  4126  1998 12151   102], shape=(128,), dtype=int32)
tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [ ]:
# Loome uue, treenitava mudeli
model = Sequential()
# Kõigepealt kaks sisendit tokeniseerijalt
model.add(Input(shape=(max_length,),dtype='int32'))
# Siis DistilBerti klassifitseerija
model.add(transformer_model)
# Lisaks mõned kihid kokkupanekuks
# LSTM, vaatab nii taha kui ette
model.add(Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)))
# Lükkame 1 dimensiooniks kokku
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
# Lõpuks 4 klassi
model.add(Dense(2, activation='softmax'))

# Alguses olevaid BERTi kaale me muuta ei taha, need on suurtel andmetel õpitud
for layer in model.layers[:3]:
  layer.trainable = False

model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_distil_bert_model (TFDist TFBaseModelOutput(last_hi 66362880  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128, 100)          327600    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_20 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 102       
Total params: 66,695,632
Trainable params: 5,152
Non-trainable params: 66,690,480
______________________________________

In [ ]:
X_train_bert['input_ids'].shape, X_test_bert['input_ids'].shape

(TensorShape([867, 128]), TensorShape([217, 128]))

In [ ]:
model.fit([X_train_bert['input_ids'],X_train_bert['attention_mask']],y_train_vec,batch_size=128,epochs=40,
        validation_data=([X_test_bert['input_Vids'],X_test_bert['attention_mask']],y_test_vec))

Epoch 1/40
7/7 [==============================] - 216s 31s/step - loss: 0.6971 - acc: 0.5375 - val_loss: 0.7038 - val_acc: 0.5392
Epoch 2/40
7/7 [==============================] - 217s 31s/step - loss: 0.7033 - acc: 0.5467 - val_loss: 0.7038 - val_acc: 0.5392
Epoch 3/40
7/7 [==============================] - 215s 31s/step - loss: 0.6944 - acc: 0.5433 - val_loss: 0.6978 - val_acc: 0.5438
Epoch 4/40
7/7 [==============================] - 215s 31s/step - loss: 0.6839 - acc: 0.5559 - val_loss: 0.6986 - val_acc: 0.5346
Epoch 5/40
7/7 [==============================] - 218s 31s/step - loss: 0.6781 - acc: 0.5917 - val_loss: 0.7058 - val_acc: 0.5392
Epoch 6/40
7/7 [==============================] - 215s 31s/step - loss: 0.6806 - acc: 0.5813 - val_loss: 0.6998 - val_acc: 0.5392
Epoch 7/40
7/7 [==============================] - 218s 31s/step - loss: 0.6861 - acc: 0.5606 - val_loss: 0.6986 - val_acc: 0.5392
Epoch 8/40
7/7 [==============================] - 215s 31s/step - loss: 0.6808 - acc: 0.57